<h1> Exploring and cleaning example dataset </h1>
<h3>IMPORT:</h3>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from numpy import loadtxt
from numpy import sort
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

from matplotlib import cbook
from matplotlib import cm
from matplotlib.colors import LightSource


print('lib-s imported')

lib-s imported


In [2]:
df = pd.read_csv('large-data-NO-GIT/US_Accidents_June20.csv', nrows=100000)

In [3]:
df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [4]:
df['Weather_Condition'].describe()

count     98396
unique       43
top       Clear
freq      57257
Name: Weather_Condition, dtype: object

In [5]:
df['Visibility(mi)'].describe()

count    98154.000000
mean         9.338446
std          1.886060
min          0.100000
25%         10.000000
50%         10.000000
75%         10.000000
max         80.000000
Name: Visibility(mi), dtype: float64

In [ ]:
df['ID'].describe()

In [ ]:
corr1 = df.corr()

In [ ]:
sns.heatmap(corr1, 
        xticklabels=corr1.columns,
        yticklabels=corr1.columns)

In [ ]:
columns = df.columns

In [ ]:
columns1 = ['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight']


<h1> Findinf fiture importance for df data </h1>

Split data into X(features) and y(target)

<h3>experimenting here with how many to use:</h3>

In [ ]:
X = df[['Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Speed(mph)',
       'Precipitation(in)']]
X1 = df[['Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]
X2 = df[['Temperature(F)', 'Pressure(in)']]

X3 = df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)']]

y = df[['Severity']]
# fit model no training data
model = XGBClassifier()
model.fit(X, y.values.ravel())
# plot feature importance
plot_importance(model)
pyplot.show()


Fit model no training data

In [ ]:
model = XGBClassifier()
model.fit(X2, y.values.ravel())

Plot feature importance

In [ ]:
plot_importance(model)
pyplot.show()

split data into train and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.33, random_state=7)

fit model on all training data

In [ ]:
model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=1.5, learning_rate=0.02,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=600, n_jobs=4, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, subsample=0.6)

In [ ]:
model.fit(X_train, y_train.values.ravel())

make predictions for test data and evaluate

In [ ]:
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Fit model using each importance as a threshold

In [ ]:
thresholds = sort(model.feature_importances_)
for thresh in thresholds:
	# select features using threshold
	selection = SelectFromModel(model, threshold=thresh, prefit=True)
	select_X_train = selection.transform(X_train)
	# train model
	selection_model = XGBClassifier()
	selection_model.fit(select_X_train, y_train.values.ravel())
	# eval model
	select_X_test = selection.transform(X_test)
	predictions = selection_model.predict(select_X_test)
	accuracy = accuracy_score(y_test, predictions)
	print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))
    

In [ ]:
X3 = X3.dropna()
z = X3[['Pressure(in)']]
nrows, ncols = z.shape
x = np.linspace(X3[['Temperature(F)']].min(), X3[['Temperature(F)']].max(), ncols)
y = np.linspace(X3[['Humidity(%)']].min(), X3[['Humidity(%)']].max(), nrows)
x, y = np.meshgrid(x, y)

ValueError: input operand has more dimensions than allowed by the axis remapping